In [3]:
import os
import shutil
import glob

def consolidate_images(base_dir):
    # Dynamically list all experimental folders using glob
    folders = [f for f in glob.glob(os.path.join(base_dir, '*/')) if os.path.isdir(f)]
    folders.sort()  # Optional: sort to ensure consistent processing order

    # Derive the new directory name from the common prefix
    common_prefix = os.path.commonprefix([os.path.basename(os.path.normpath(f)) for f in folders])
    new_dir = os.path.join(base_dir, common_prefix)

    # Create the new directory for consolidated images
    os.makedirs(new_dir, exist_ok=True)

    # Dynamically list all Pos folders from the first experimental folder
    first_folder_path = folders[0]
    pos_folders = [d for d in os.listdir(first_folder_path) if os.path.isdir(os.path.join(first_folder_path, d))]
    pos_folders.sort()  # Optional: sort to ensure consistent processing order

    # Create subfolders for each Pos
    for pos in pos_folders:
        pos_folder_path = os.path.join(new_dir, pos)
        os.makedirs(pos_folder_path, exist_ok=True)

    # Initialize counters for Cy5, GFP, DAPI images, metadata, and display_and_comments.txt files
    cy5_counter = {pos: 0 for pos in pos_folders}
    gfp_counter = {pos: 0 for pos in pos_folders}
    dapi_counter = {pos: 0 for pos in pos_folders}
    metadata_counter = {pos: 0 for pos in pos_folders}
    comments_counter = 0

    # Function to count images in a folder
    def count_images(folder_path):
        if not os.path.exists(folder_path):
            return 0
        return len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

    # Move images, metadata, and display_and_comments.txt, and update counters
    for folder in folders:
        for pos in pos_folders:
            current_pos_folder_path = os.path.join(folder, pos)
            
            if not os.path.exists(current_pos_folder_path):
                print(f"Warning: {current_pos_folder_path} does not exist.")
                continue
            
            images = sorted(os.listdir(current_pos_folder_path))
            
            for image in images:
                old_image_path = os.path.join(current_pos_folder_path, image)
                image_lower = image.lower()  # Make the image name lowercase
                
                if 'cy5' in image_lower:
                    prefix, ext = os.path.splitext(image)
                    parts = prefix.split('_')
                    new_image_name = f'img_{cy5_counter[pos]:09d}_{parts[2]}_{parts[3]}{ext}'
                    cy5_counter[pos] += 1
                elif 'gfp' in image_lower:
                    prefix, ext = os.path.splitext(image)
                    parts = prefix.split('_')
                    new_image_name = f'img_{gfp_counter[pos]:09d}_{parts[2]}_{parts[3]}{ext}'
                    gfp_counter[pos] += 1
                elif 'dapi' in image_lower:
                    prefix, ext = os.path.splitext(image)
                    parts = prefix.split('_')
                    new_image_name = f'img_{dapi_counter[pos]:09d}_{parts[2]}_{parts[3]}{ext}'
                    dapi_counter[pos] += 1
                elif image == 'metadata.txt':
                    # Move and rename metadata.txt to avoid overwriting
                    new_image_name = f'metadata_{metadata_counter[pos]:03d}.txt'
                    metadata_counter[pos] += 1
                else:
                    continue
                
                new_image_path = os.path.join(new_dir, pos, new_image_name)
                
                try:
                    shutil.move(old_image_path, new_image_path)
                except Exception as e:
                    print(f"Error moving {old_image_path} to {new_image_path}: {e}")

            # Move display_and_comments.txt and rename it
            comments_file_path = os.path.join(folder, 'display_and_comments.txt')
            if os.path.exists(comments_file_path):
                new_comments_path = os.path.join(new_dir, f'display_and_comments_{comments_counter:03d}.txt')
                comments_counter += 1
                try:
                    shutil.move(comments_file_path, new_comments_path)
                except Exception as e:
                    print(f"Error moving {comments_file_path} to {new_comments_path}: {e}")

            # If the Pos folder is empty after moving images, delete it
            if not os.listdir(current_pos_folder_path):
                try:
                    os.rmdir(current_pos_folder_path)
                    print(f"Deleted empty folder: {current_pos_folder_path}")
                except Exception as e:
                    print(f"Error deleting folder {current_pos_folder_path}: {e}")

        # If the main folder is empty after moving the display_and_comments.txt file, delete it
        if not os.listdir(folder):
            try:
                os.rmdir(folder)
                print(f"Deleted empty folder: {folder}")
            except Exception as e:
                print(f"Error deleting folder {folder}: {e}")

    # Check and count images in final folders
    print("\nChecking final consolidated folders:")
    for pos in pos_folders:
        pos_folder_path = os.path.join(new_dir, pos)
        count = count_images(pos_folder_path)
        print(f"Images in final {pos_folder_path}: {count}")

    print("Renaming, moving, and cleanup completed.")

# Example usage
base_dir = '../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/microscope/083024-ABCDEFGH-RT/'
consolidate_images(base_dir)


Deleted empty folder: ../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/microscope/083024-ABCDEFGH-RT/3ulTMB-0p5ulDNA_all50nM_4/Pos0
Deleted empty folder: ../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/microscope/083024-ABCDEFGH-RT/3ulTMB-0p5ulDNA_all50nM_4/Pos1
Deleted empty folder: ../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/microscope/083024-ABCDEFGH-RT/3ulTMB-0p5ulDNA_all50nM_4/Pos2
Deleted empty folder: ../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/microscope/083024-ABCDEFGH-RT/3ulTMB-0p5ulDNA_all50nM_4/Pos3
Deleted empty folder: ../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/microscope/083024-ABCDEFGH-RT/3ulTMB-0p5ulDNA_all50nM_4/Pos4
Deleted empty folder: ../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/microscope/083024-ABCDEFGH-RT/3ulTMB-0p5ulDNA_all50nM_4/Pos5
Deleted empty fo